# Building a Local MCP Client with LlamaIndex Agent

This Jupyter notebook walks you through creating a **local MCP (Model Context Protocol) client** that can chat with a database through tools exposed by an MCP server—completely on your machine. Follow the cells in order for a smooth, self‑contained tutorial.


# Architecture and Benefits of MCP Client & Agent Separation
╔══════════════════════════════════════════════════════════════════════════════╗
║                  MCP Client & Agent: Separation of Concerns                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  ┌──────────────┐        Tool Calls        ┌──────────────┐                  ║
║  │   Agent      │ ───────────────────────► │  MCP Client  │                  ║
║  │ (LLM+Logic)  │                          │ (Tool Proxy) │                  ║
║  └──────────────┘   ◄────────────────────  └──────────────┘                  ║
║         ▲        Tool Results/Responses           ▲                          ║
║         │                                         │                          ║
║   User Input/Chat                          MCP Server/Tools                  ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║ • The **Agent** (LLM+Logic) interprets user intent and decides which tools   ║
║   to call, but does not know tool implementation details.                    ║
║ • The **MCP Client** discovers, registers, and communicates with tools       ║
║   exposed by the MCP server, abstracting away tool endpoints and protocols.  ║
║ • This separation enables:                                                   ║
║    - Maintainability: Add/update tools in MCP layer, not agent logic.        ║
║    - Extensibility: Swap agent or tool backend independently.                ║
║    - Security: Agent never has direct access to tool implementations.        ║
║    - Transparency: All tool calls are logged for quality/debugging.          ║
╚══════════════════════════════════════════════════════════════════════════════╝

In [1]:
import nest_asyncio
nest_asyncio.apply()

## 2  Setup a local LLM

In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

## 3  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), and list the available tools.

In [3]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) # you can also pass list of allowed tools

In [4]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

add_data Add new data to the people table.

Args:
    name (str): Name of the person.
    age (int): Age of the person.
    profession (str): Profession of the person.
Returns:
    bool: True if data was added successfully, False otherwise.
Example:
    >>> add_data('Roger Federer', 45, 'Professional Tennis Player')
    True

read_data Read data from the people table using a SQL SELECT query and return as a list of dicts.

Args:
    query (str, optional): SQL SELECT query. Defaults to "SELECT * FROM people".
        Examples:
        - "SELECT * FROM people"
        - "SELECT name, age FROM people WHERE age > 25"
        - "SELECT * FROM people ORDER BY age DESC"

Returns:
    list: List of dictionaries containing the query results.
          For default query, dict format is {"id": ..., "name": ..., "age": ..., "profession": ...}

Example:
    >>> # Read all records
    >>> read_data()
    [
        {"id": 1, "name": "John Doe", "age": 30, "profession": "Engineer"},
        {"id": 2, 

## 3  Define the system prompt
This prompt steers the LLM when it needs to decide how and when to call tools.

In [5]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with Our Database.

Always use the available tools to answer user questions. Do not make up information or answer from memory—only use the results returned by the tools.
"""

## 4  Helper function: `get_agent()`
Creates a `FunctionAgent` wired up with the MCP tool list and your chosen LLM.

In [6]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        llm = Settings.llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

## 5  Helper function: `handle_user_message()`
Streams intermediate tool calls (for transparency) and returns the final response.

In [7]:
from llama_index.core.agent.workflow import (
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and isinstance(event, ToolCall):
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
            # Print the actual tool call for LLM quality assessment
            print(f"[MCP AGENT TOOL CALL] Tool: {event.tool_name}, Arguments: {event.tool_kwargs}")
        elif verbose and isinstance(event, ToolCallResult):
            print(f"Tool {event.tool_name} returned {event.tool_output}")
    response = await handler
    return str(response)

## 6  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), build the agent, and setup agent context.

In [8]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [ ]:
# Run the agent!
while True:
    #user_input = input("Enter your message: (type exit to quit)")
    print("Enter your message (type 'exit' to quit):")
    user_input = input("> ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

Enter your message (type 'exit' to quit):
